## Dataset 1: Datos para la predicción del rendimiento en cultivos

In [1]:
%cd PRO502/

/workspace/PRO502


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, FloatType,BooleanType


spark = ( SparkSession.builder.appName("PRO502").master("spark://spark-master:7077").getOrCreate())

sc = spark.sparkContext



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/23 09:40:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
26/01/23 09:40:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


KeyboardInterrupt: 

26/01/23 09:40:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
main_schema = StructType([
    StructField("Crop",StringType(),True),
    StructField("Region",StringType(),True),
    StructField("Soil_Type",StringType(),True),
    StructField("Soil_pH",FloatType(),True),
    StructField("Rainfall_mm",FloatType(),True),
    StructField("Temperature_C",FloatType(),True),
    StructField("Humidity_pct",FloatType(),True),
    StructField("Fertilizer_Used_Kg",FloatType(),True),
    StructField("Irrigation",StringType(),True),
    StructField("Pesticides_Used_kg",FloatType(),True),
    StructField("Planting_Density",FloatType(),True),
    StructField("Previus_Crop",StringType(),True),
    StructField("Yield_ton_per_ha",FloatType(),True)
])

df_crop = (spark.read.format("csv")
      .option("header","true")
      .schema(main_schema)
      .load("crop_yield_dataset.csv"))


df_sel = df_crop.select("Crop","Region","Temperature_C","Rainfall_mm","Irrigation","Yield_ton_per_ha")



In [ ]:
df_renamed = (df_sel
    .withColumnRenamed("Temperature_C", "Temperatura")
    .withColumnRenamed("Rainfall_mm", "Lluvia")
    .withColumnRenamed("Yield_ton_per_ha", "Rendimiento")
)
df_renamed.show(5)

+------+--------+-----------+------+----------+-----------+
|  Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+------+--------+-----------+------+----------+-----------+
| Maize|Region_C|       19.7|1485.4|      Drip|     101.48|
|Barley|Region_D|       29.1| 399.4| Sprinkler|     127.39|
|  Rice|Region_C|       30.5| 980.9| Sprinkler|      68.99|
| Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
| Maize|Region_D|       20.4| 744.6|      Drip|     118.71|
+------+--------+-----------+------+----------+-----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col

df_filtered = df_renamed.filter(
    (col("Crop") == "Maize") & 
    (col("Temperatura") > 5)
)

df_filtered.show(5)

+-----+--------+-----------+------+----------+-----------+
| Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+-----+--------+-----------+------+----------+-----------+
|Maize|Region_C|       19.7|1485.4|      Drip|     101.48|
|Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
|Maize|Region_D|       20.4| 744.6|      Drip|     118.71|
|Maize|Region_C|       32.4| 846.1|      None|      162.2|
|Maize|Region_C|       18.3| 209.5|     Flood|     116.72|
+-----+--------+-----------+------+----------+-----------+
only showing top 5 rows



In [ ]:
df_final = (df_sel
    .withColumnRenamed("Temperature_C", "Temperatura")
    .withColumnRenamed("Rainfall_mm", "Lluvia")
    .withColumnRenamed("Yield_ton_per_ha", "Rendimiento")
    .filter((col("Crop") == "Maize") & (col("Temperatura") > 25))
)

# Visualizamos el resultado final
df_final.show(5)

+-----+--------+-----------+------+----------+-----------+
| Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+-----+--------+-----------+------+----------+-----------+
|Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
|Maize|Region_C|       32.4| 846.1|      None|      162.2|
|Maize|Region_A|       26.6| 362.5| Sprinkler|      95.23|
|Maize|Region_C|       33.7|1193.3|      None|     110.57|
|Maize|Region_C|       27.8| 695.2|     Flood|     143.84|
+-----+--------+-----------+------+----------+-----------+
only showing top 5 rows



## Dataset 2: Lugares famosos del mundo

In [ ]:
main_schema = StructType([
    StructField("Place_Name",StringType(),True),
    StructField("Country",StringType(),True),
    StructField("City",StringType(),True),
    StructField("Annual_Visitors_Millions",FloatType(),True),
    StructField("Type",StringType(),True),
    StructField("Unsesco_World_Heritage",StringType(),True),
    StructField("Year_Built",StringType(),True),
    StructField("Entry_Fee_USD",StringType(),True),
    StructField("Best_Visit_Month",StringType(),True),
    StructField("Region",StringType(),True),
    StructField("Tourism_Revenue_Million_USD",IntegerType(),True),
    StructField("Average_Visit_Duration_Hours",FloatType(),True),
    StructField("Famous_For",StringType(),True),
])

df = (spark.read.format("csv")
      .option("header","true")
      .schema(main_schema)
      .load("world_famous_places_2024.csv"))

df_base = df.select("Place_Name","Country","Unsesco_World_Heritage","Entry_Fee_USD","Annual_Visitors_Millions")

In [ ]:
df_es = (df_base
    .withColumnRenamed("Place_Name", "Lugar")
    .withColumnRenamed("Unsesco_World_Heritage", "Es_UNESCO")
    .withColumnRenamed("Entry_Fee_USD", "Precio_Entrada")
    .withColumnRenamed("Annual_Visitors_Millions", "Visitantes_Millones")
)
df_es.show()

+--------------------+-------------+---------+--------------+-------------------+
|               Lugar|      Country|Es_UNESCO|Precio_Entrada|Visitantes_Millones|
+--------------------+-------------+---------+--------------+-------------------+
|        Eiffel Tower|       France|       No|            35|                7.0|
|        Times Square|United States|       No|             0|               50.0|
|       Louvre Museum|       France|      Yes|            22|                8.7|
| Great Wall of China|        China|      Yes|            10|               10.0|
|           Taj Mahal|        India|      Yes|            15|                7.5|
|           Colosseum|        Italy|      Yes|            18|               7.65|
|   Statue of Liberty|United States|      Yes|            25|                4.3|
|  Sydney Opera House|    Australia|      Yes|            49|                8.2|
|        Machu Picchu|         Peru|      Yes|            70|                1.5|
|      Forbidden

In [ ]:
df_fil = df_es.filter(
        (col("Es_UNESCO").contains("Yes")) & 
        (col("Precio_Entrada") <= 20)
    )

df_fil.show(5)

+--------------------+-------+---------+--------------+-------------------+
|               Lugar|Country|Es_UNESCO|Precio_Entrada|Visitantes_Millones|
+--------------------+-------+---------+--------------+-------------------+
| Great Wall of China|  China|      Yes|            10|               10.0|
|           Taj Mahal|  India|      Yes|            15|                7.5|
|           Colosseum|  Italy|      Yes|            18|               7.65|
|      Forbidden City|  China|      Yes|             8|                9.0|
|Notre-Dame Cathedral| France|      Yes|             0|               13.0|
+--------------------+-------+---------+--------------+-------------------+
only showing top 5 rows



## Dataset 3: Registro turístico de Castilla y León

In [ ]:
main_schema = StructType([
    StructField("establecimiento", StringType(), True),
    StructField("n_registro", StringType(), True),
    StructField("codigo", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("categoria", StringType(), True),
    StructField("especialidades", StringType(), True),
    StructField("clase", StringType(), True),
    StructField("nombre", StringType(), True),
    StructField("direccion", StringType(), True),
    StructField("c_postal", StringType(), True), 
    StructField("provincia", StringType(), True),
    StructField("municipio", StringType(), True),
    StructField("localidad", StringType(), True),
    StructField("nucleo", StringType(), True),
    StructField("telefono_1", StringType(), True),
    StructField("telefono_2", StringType(), True),
    StructField("telefono_3", StringType(), True),
    StructField("email", StringType(), True),
    StructField("web", StringType(), True),
    StructField("q_calidad", StringType(), True),
    StructField("posada_real", StringType(), True),
    StructField("plazas", IntegerType(), True),
    StructField("gps_longitud", FloatType(), True),
    StructField("gps_latitud", FloatType(), True),
    StructField("accesible_a_personas_con_discapacidad", StringType(), True),
    StructField("column_27", StringType(), True),
    StructField("posicion", StringType(), True)
])

df = (spark.read.format("csv")
      .option("header","true")
      .schema(main_schema)
      .option("sep", ";")
      .load("registro-de-turismo-de-castilla-y-leon.csv"))


df_contactos = df.select("nombre", "tipo", "provincia", "web", "email")

df_contactos.show(5)

+--------------------+--------------------+---------+--------------------+--------------------+
|              nombre|                tipo|provincia|                 web|               email|
+--------------------+--------------------+---------+--------------------+--------------------+
|BERNARDO MORO MEN...|Profesional de Tu...| Asturias|                NULL|bernardomoro@hotm...|
|        LA SASTRERÍA|Casa Rural de Alq...|    Ávila|www.lasastreriade...|                NULL|
|         LAS HAZANAS|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
| LA CASITA DEL PAJAR|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
|            MARACANA|                 Bar|    Ávila|                NULL|emo123anatoliev@g...|
+--------------------+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df_limpio = (df_contactos
    .withColumnRenamed("nombre", "nombre_establecimiento")
    .withColumnRenamed("tipo", "categoria_actividad")
    .withColumnRenamed("web", "sitio_web")
    .withColumnRenamed("email", "correo_electronico")
)

df_limpio.show(5)

+----------------------+--------------------+---------+--------------------+--------------------+
|nombre_establecimiento| categoria_actividad|provincia|           sitio_web|  correo_electronico|
+----------------------+--------------------+---------+--------------------+--------------------+
|  BERNARDO MORO MEN...|Profesional de Tu...| Asturias|                NULL|bernardomoro@hotm...|
|          LA SASTRERÍA|Casa Rural de Alq...|    Ávila|www.lasastreriade...|                NULL|
|           LAS HAZANAS|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
|   LA CASITA DEL PAJAR|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
|              MARACANA|                 Bar|    Ávila|                NULL|emo123anatoliev@g...|
+----------------------+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df_final = df_limpio.filter(
    (col("provincia") == "Burgos") & 
    (col("categoria_actividad").like("%Bodegas%")) & 
    (col("sitio_web").isNotNull()) & 
    (col("sitio_web") != "")
)


df_final.show()

+----------------------+--------------------+---------+--------------------+--------------------+
|nombre_establecimiento| categoria_actividad|provincia|           sitio_web|  correo_electronico|
+----------------------+--------------------+---------+--------------------+--------------------+
|        BODEGAS TARSUS|g - Bodegas y los...|   Burgos|  www.tarsusvino.com|                NULL|
|  BODEGAS DOMINIO D...|g - Bodegas y los...|   Burgos|www.dominiodecair...|bodegas@dominiode...|
|    TERRITORIO LUTHIER|g - Bodegas y los...|   Burgos|territorioluthier...|luthier@territori...|
|    BODEGA COVARRUBIAS|g - Bodegas y los...|   Burgos| http://valdable.com|   info@valdable.com|
|  BODEGAS PASCUAL, ...|g - Bodegas y los...|   Burgos|222.bodegaspascua...|export@bodegaspas...|
|   BODEGAS VINUM VITAE|g - Bodegas y los...|   Burgos|      www.avañate.es|vinum.vitae.bodeg...|
|  VIÑEDOS Y BODEGAS...|g - Bodegas y los...|   Burgos|     www.ferratus.es|administracion@fe...|
|  BODEGAS Y VIÑEDOS